## Setting up the packages and relevant libraries

In [1]:
using NeuralPDE, Lux, Plots, Random, Distributions, Optim , OptimizationOptimJL
using IntegralsCubature, FinancialToolbox
using ModelingToolkit: Interval
Random.seed!(0)




TaskLocalRNG()

## Analytical solution function_ Call_pricer

In [2]:
function call_pricer(x,t)
    d1(x,t) = (log(x./K) .+ (r .+ 0.5*σ.^2).*(T-t))./(σ*sqrt(T-t))
    d2(x,t) = d1.(x,t) .- σ*sqrt(T-t)
    return x.*normcdf.(d1.(x,t)) .- exp(-r*(T-t))*K*normcdf.(d2.(x,t)) 
end

call_pricer (generic function with 1 method)

## Parameters

In [3]:

σ = 0.2 
r = 0.1
K = 100 # strike is 100
T = 1 #one year 
Smin = 80
Smax = 120 # stock range make it 20% in-the-money and out-the-money
#scaling_factor = 1
scaling_factor = 0.0555
boundary_func(s) = max(s - K,0)*scaling_factor

boundary_func (generic function with 1 method)

## Setting up the PDE for discretization - making it ready for PINN

In [4]:
# 2D PDE in (t, s)
@parameters t s
@variables U(..)
Dt = Differential(t)
Ds = Differential(s)
Dss = Differential(s)^2

eq = Dt(U(t, s)) + r*s*Ds(U(t, s)) + 1/2*σ^2*s^2*Dss(U(t, s)) - r*U(t, s) ~ 0

# Boundary Conditions
boundary_conditions = [U(T, s) ~ boundary_func(s)]

# Problem Domain
domains = [t ∈ Interval(0, T), s ∈ Interval(Smin, Smax)]

2-element Vector{Symbolics.VarDomainPairing}:
 Symbolics.VarDomainPairing(t, 0..1)
 Symbolics.VarDomainPairing(s, 80..120)

## Transforming into optimization problem - ready to be optimized

In [5]:
dim = 2
chain = Lux.Chain(Dense(dim, 16, Lux.σ), Dense(16, 16, Lux.σ), Dense(16, 1))
ps = Lux.setup(Random.default_rng(), chain)[1]

# Transform PDESystem into OptimizationProblem using PINN methodology
strategy = QuadratureTraining()
discretization_algo = PhysicsInformedNN(chain, strategy, init_params=ps)
@named pde_system = PDESystem(eq, boundary_conditions, domains, [t, s], [U(t, s)])
optim_prob = discretize(pde_system, discretization_algo)

OptimizationProblem. In-place: true
u0: ComponentVector{Float32}(layer_1 = (weight = Float32[-0.5096706 0.42996323; -0.051686242 -0.19706658; … ; -0.23860326 5.093088f-6; 0.5504109 -0.03434386], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = Float32[0.15251705 0.39284882 … -0.022368534 0.20915501; -0.2943246 0.08905726 … 0.2225429 -0.1315318; … ; -0.42614084 -0.41936278 … -0.3245835 -0.14442606; 0.047960266 0.27465823 … 0.005657302 0.058415208], bias = Float32[0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = Float32[0.3952902 0.19821757 … -0.5427456 0.17304797], bias = Float32[0.0;;]))

## Getting trial solution - optimal weights of NN

In [ ]:
import Pkg
Pkg.add("TickTock")

In [ ]:
using TickTock

In [6]:
#tick()
callback = function (p,l)
    println("Current loss is: $l")
    return false
end

opt = BFGS()
result = Optimization.solve(optim_prob, opt, callback = callback, maxiters=2500, reltol=1e-6)
result.original
phi = discretization_algo.phi
#tock()

Current loss is: 4.687473973001896
Current loss is: 2.973390552337211
Current loss is: 2.7632212951875506
Current loss is: 2.7624385008634933
Current loss is: 2.7541601325662746
Current loss is: 2.74879715278585
Current loss is: 2.7433083935079385
Current loss is: 2.7200636915466485
Current loss is: 2.71429309754764
Current loss is: 2.7073394377728657
Current loss is: 2.7045727213707544
Current loss is: 2.7019920998392637
Current loss is: 2.6987228833247663
Current loss is: 2.672399451397725
Current loss is: 2.632315634352009
Current loss is: 2.5553910712120733
Current loss is: 2.511773758554655
Current loss is: 2.4877773033025536
Current loss is: 2.3165185099196983
Current loss is: 2.283507735706717
Current loss is: 2.0518901952121107
Current loss is: 1.9786183905311754
Current loss is: 1.9172173836126272
Current loss is: 1.8004008630887998
Current loss is: 1.7756612747276574
Current loss is: 1.7729980330854365
Current loss is: 1.521000581381945
Current loss is: 1.4027821002382912
Cur

Current loss is: 0.015331731831246012
Current loss is: 0.015293642841417751
Current loss is: 0.015225274686104125
Current loss is: 0.015148347567497063
Current loss is: 0.015073357865541797
Current loss is: 0.01498902579768249
Current loss is: 0.014914213703800496
Current loss is: 0.014786239543312435
Current loss is: 0.014722919317768061
Current loss is: 0.014696539453970738
Current loss is: 0.014617521932018547
Current loss is: 0.014474157685575854
Current loss is: 0.014336593964470885
Current loss is: 0.01428005719659107
Current loss is: 0.01405615076471131
Current loss is: 0.013614106114965541
Current loss is: 0.013449115636102483
Current loss is: 0.013389023414119103
Current loss is: 0.01325615285563769
Current loss is: 0.013256156651848011
Current loss is: 0.013250964448592903
Current loss is: 0.013030736730993617
Current loss is: 0.012688680669149664
Current loss is: 0.01251090336491696
Current loss is: 0.01238659412891578
Current loss is: 0.011969441359772612
Current loss is: 0

Current loss is: 0.0028051454528943104
Current loss is: 0.0028048632462473397
Current loss is: 0.0028048645793580474
Current loss is: 0.0028023976142055687
Current loss is: 0.00280107928456894
Current loss is: 0.00280062310585446
Current loss is: 0.0028006229221644845
Current loss is: 0.0027988402961488627
Current loss is: 0.002796982429465349
Current loss is: 0.0027950139758835574
Current loss is: 0.002791569871350946
Current loss is: 0.0027882248439960807
Current loss is: 0.0027860519659933724
Current loss is: 0.0027835051590213004
Current loss is: 0.0027822772350974546
Current loss is: 0.0027814646444665374
Current loss is: 0.0027780244482172663
Current loss is: 0.0027751534513184187
Current loss is: 0.0027741676508085543
Current loss is: 0.002771784468620187
Current loss is: 0.0027679350921830095
Current loss is: 0.0027671156406160044
Current loss is: 0.0027667650088415018
Current loss is: 0.002764352195403402
Current loss is: 0.002761966799056527
Current loss is: 0.002761618046478

NeuralPDE.Phi{Chain{NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{Dense{true, typeof(NNlib.sigmoid_fast), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}, Dense{true, typeof(NNlib.sigmoid_fast), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}, Dense{true, typeof(identity), typeof(Lux.glorot_uniform), typeof(Lux.zeros32)}}}}, NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}}}}(Chain(), (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()))

In [ ]:
dx = 0.5
x_domain = Smin:dx:Smax
S_domain = collect(x_domain)

times = [0, 0.25, 0.5, 1]
#plots = Array{Plots.Plot{Plots.GRBackend},1}()


u_predict1 = [phi([times[1], current_x],result.u) for current_x in x_domain]
u_predict1 = reduce(vcat, u_predict1)/scaling_factor
u_true1 = call_pricer(S_domain, times[1])

u_predict2 = [phi([times[2], current_x],result.u) for current_x in x_domain]
u_predict2 = reduce(vcat, u_predict2)/scaling_factor
u_true2 = call_pricer(S_domain, times[2])

u_predict3 = [phi([times[3], current_x],result.u) for current_x in x_domain]
u_predict3 = reduce(vcat, u_predict3)/scaling_factor
u_true3 = call_pricer(S_domain, times[3])

u_predict4 = [phi([times[4], current_x],result.u) for current_x in x_domain]
u_predict4 = reduce(vcat, u_predict4)/scaling_factor
u_true4 = max.(S_domain .- K, 0)

#for i in 1:length(times)
    #current_t = times[i]
    #u_predict = [phi([current_t, current_x],result.u) for current_x in x_domain]
    #u_predict = reduce(vcat, u_predict)/scaling_factor
    #u_predict = max.(u_predict,0)
    #u_true = call_pricer(S_domain, current_t)
    #push!(plots, plot(S_domain, [u_true, u_predict], label= ["Analytical" "PINN"], title="t = $(current_t)",legend=:topleft,
            #xlabel = "Stock prices", ylabel = "Option prices", grid=:false,framestyle = :box, widen=:false))
    
#end

#plot(plots[1], plots[2], plots[3], plots[4], layout = 4)




In [ ]:
using PyPlot
using Makie # to enable save function 

#figure(1);
fig = figure(figsize=(10, 10), dpi=1200);
#subplots_adjust(hspace=0.5,wspace=0.9)

#subplot(221) ;  PyPlot.plot(x_domain,u_predict1,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true1,color = "magenta", label = "Analytical");
#PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
#; PyPlot.legend(loc="upper left")

#subplot(222) ;  PyPlot.plot(x_domain,u_predict4,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true4,color = "magenta", label = "Analytical");
#PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
#; PyPlot.legend(loc="upper left")

subplot(221) ;  PyPlot.plot(x_domain,u_predict1,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true1,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(222) ;  PyPlot.plot(x_domain,u_predict2,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true2,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.25",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(223) ;  PyPlot.plot(x_domain,u_predict3,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true3,color = "magenta", label = "Analytical");
PyPlot.title("Time = 0.5",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(224) ;  PyPlot.plot(x_domain,u_predict4,color = "blue",label = "PINN"); PyPlot.plot(x_domain,u_true4,color = "magenta", label = "Analytical");
PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

fig
save("./Fig_21.png", fig)


In [ ]:
using PyPlot

#figure(1);
fig = figure(figsize=(10, 10), dpi=1200);
#subplots_adjust(hspace=0.5,wspace=0.9)
subplot(221) ;  PyPlot.plot(x_domain, abs.(u_predict1.- u_true1),color = "magenta",label = "Absolute error");
PyPlot.title("Time = 0",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(222) ;  PyPlot.plot(x_domain, abs.(u_predict2.- u_true2) ,color = "magenta",label = "Absolute error");
PyPlot.title("Time = 0.25",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(223) ;  PyPlot.plot(x_domain, abs.(u_predict3.- u_true3) ,color = "magenta",label = "Absolute error"); 
PyPlot.title("Time = 0.5",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

subplot(224) ;  PyPlot.plot(x_domain, abs.(u_predict4.- u_true4) ,color = "magenta",label = "Absolute error"); 
PyPlot.title("Time = 1",fontweight="bold") ; xlabel("S") ; ylabel("Call prices")
; PyPlot.legend(loc="upper left")

fig
save("./Fig_2.png", fig)


## Error plot

In [ ]:

using Plots

plots = Array{Plots.Plot{Plots.GRBackend},1}()

for i in 1:length(times)
    current_t = times[i]
    u_predict = [phi([current_t, current_x],result.u) for current_x in S_domain]
    u_predict = reduce(vcat, u_predict)/scaling_factor
    u_true = call_pricer(S_domain, current_t)
    error = abs.(u_true - u_predict)
    push!(plots, Plots.plot(S_domain, error, label=:false ,legend=:topleft, title="t = $(current_t)",
            xlabel = "S", ylabel = "Absolute error"))
    
end

Plots.plot(plots[1], plots[2], plots[3], plots[4], layout = 4, framestyle=:box, dpi=150, lw = 2)



## Calculating vol

In [ ]:
function BS_vol(S0, K, r, T, price)
    f(x) = call_pricer(S0,0) - price
    vol = find_zero(f, 0.3)
    return vol
end

In [ ]:
u_predict = [phi([0, current_x],result.u) for current_x in S_domain]
u_predict = reduce(vcat, u_predict)/scaling_factor

In [ ]:
BS_vol(S_domain, K, r, T, u_predict)

In [ ]:
import Pkg
Pkg.add("Roots")

In [ ]:
using Roots